In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

# Load the dataset
df = pd.read_csv("C:/Users/Jabasingh Daniel/Desktop/EGDK/Dataset/cob_py.csv")

In [5]:
df['python_code'] = df['python_code'].astype(str)
summary = df['python_code'].values
summary = ['<start> ' + text + ' <end>' for text in summary]

In [21]:
#summary 

In [12]:
#### import pandas as pd

# Assuming df is your DataFrame with the 'summary' column
df['python_code'] = '<start> ' + df['python_code'] + ' <end>'


In [14]:
tokenizer_cobol = Tokenizer(filters='')
tokenizer_cobol.fit_on_texts(df['cobol_code'])
tokenizer_summary = Tokenizer(filters='')
tokenizer_summary.fit_on_texts(df['python_code'])

# Define the vocabulary sizes
vocab_size_cobol = len(tokenizer_cobol.word_index) + 1
vocab_size_summary = len(tokenizer_summary.word_index) + 1

In [15]:
#tokenizer_cobol.index_word

In [16]:
#tokenizer_summary.index_word

In [22]:
# Prepare input-output pairs
X = tokenizer_cobol.texts_to_sequences(df['cobol_code'])
Y = tokenizer_summary.texts_to_sequences(df['python_code'])
# Pad sequences
max_len_cobol = max([len(seq) for seq in X])
max_len_summary = max([len(seq) for seq in Y])

X = pad_sequences(X, maxlen=max_len_cobol, padding='post')
Y = pad_sequences(Y, maxlen=max_len_summary, padding='post')

In [23]:
# Shift the target sequencesY for training
Y_input = Y[:, :-1]
Y_output = Y[:, 1:]

# Define the Seq2Seq model
latent_dim = 512

encoder_inputs = Input(shape=(max_len_cobol,))
encoder_embedding = Embedding(vocab_size_cobol, latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_len_summary-1,))
decoder_embedding = Embedding(vocab_size_summary, latent_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size_summary, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [25]:
# Compile and train the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit([X, Y[:,:-1]], Y.reshape(Y.shape[0], Y.shape[1], 1)[:,1:],batch_size=10, epochs=100, validation_split=0.2)


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.0421 - loss: 6.1786 - val_accuracy: 0.0390 - val_loss: 6.1652
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.0476 - loss: 6.0731 - val_accuracy: 0.0223 - val_loss: 6.1040
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.0223 - loss: 5.8929 - val_accuracy: 0.0390 - val_loss: 5.9128
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.0453 - loss: 5.3962 - val_accuracy: 0.0390 - val_loss: 6.0553
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.0408 - loss: 5.2898 - val_accuracy: 0.0390 - val_loss: 6.1237
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.0429 - loss: 5.1674 - val_accuracy: 0.0390 - val_loss: 6.2042
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.2375 - loss: 5.0734 - val_accuracy: 0.0390 - val_loss: 6.2624
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.0428 - loss: 4.9792 - val_accuracy: 0.0390 - val_loss: 6.3478
Epoch 9

In [26]:
# Save the model
model.save('seq2seq_model_cobo_py.keras') 

In [49]:
#testing

import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

# Assuming your test data is loaded and preprocessed similarly to the training data
df_test = pd.read_csv("C:/Users/Jabasingh Daniel/Desktop/EGDK/Dataset/cob_py.csv")
df_test = df_test.fillna('')

# Ensure all values are strings
df_test['cobol_code'] = df_test['cobol_code'].astype(str)
df_test['python_code'] = df_test['python_code'].astype(str)

X_test = tokenizer_cobol.texts_to_sequences(df_test['cobol_code'])
Y_test = tokenizer_summary.texts_to_sequences(df_test['python_code'])

# Load the trained model
model = load_model("C:/Users/Jabasingh Daniel/Desktop/EGDK/modeling/seq2seq_model_cobo_py.keras")


# Define start and end tokens
start_token = tokenizer_summary.word_index.get('<start>', 0)
end_token = tokenizer_summary.word_index.get('<end>', 0)

# Function to predict summary for a single input sequence
def predict_summary(input_seq, model, tokenizer_summary, max_len_summary):
    decoder_input = np.zeros((1, max_len_summary - 1))
    decoder_input[0, 0] = start_token
    
    for i in range(1, max_len_summary - 1):
        output_tokens = model.predict([input_seq, decoder_input])
        sampled_token_index = np.argmax(output_tokens[0, i-1, :])
        decoder_input[0, i] = sampled_token_index
        
        if sampled_token_index == end_token:
            break
    
    predicted_summary = []
    for token in decoder_input[0]:
        if token == 0:
            continue
        word = tokenizer_summary.index_word.get(token, '')
        if word == '<end>':
            break
        predicted_summary.append(word)
    
    return ' '.join(predicted_summary)

# Function to calculate sequence accuracy
def sequence_accuracy(y_true, y_pred):
    correct = 0
    total = 0
    for true_seq, pred_seq in zip(y_true, y_pred):
        true_seq = true_seq[true_seq != 0]  # Remove padding
        pred_seq = pred_seq[:len(true_seq)]  # Truncate to the length of the true sequence
        if np.array_equal(true_seq, pred_seq):
            correct += 1
        total += 1
    return correct / total

predicted_summaries = []
for i in range(len(X_test)):
    input_seq = X_test[i:i+1]
    predicted_summary = predict_summary(input_seq, model, tokenizer_summary, max_len_summary)
    predicted_summaries.append(predicted_summary)

# Tokenize predicted summaries
Y_pred = tokenizer_summary.texts_to_sequences(predicted_summaries)
Y_pred = pad_sequences(Y_pred, maxlen=max_len_summary, padding='post')

# Calculate the accuracy
accuracy = sequence_accuracy(Y_test[:, 1:], Y_pred_sequences)
print(f'Sequence Accuracy: {accuracy * 100:.2f}%')


ValueError: Unrecognized data type: x=[[[8, 2, 9, 204, 10, 2, 3, 205, 206, 11, 12]], array([[2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])] (of type <class 'list'>)

In [44]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("C:/Users/Jabasingh Daniel/Desktop/EGDK/modeling/seq2seq_model.keras")

def preprocess_input(cobol_code, tokenizer_cobol, max_len_cobol):
    # Tokenize the input COBOL code
    sequence = tokenizer_cobol.texts_to_sequences([cobol_code])
    # Pad the sequence
    padded_sequence = pad_sequences(sequence, maxlen=max_len_cobol, padding='post')
    return padded_sequence


In [45]:
def predict_summary(cobol_code, model, tokenizer_cobol, tokenizer_summary, max_len_cobol, max_len_summary):
    # Preprocess the input COBOL code
    input_seq = preprocess_input(cobol_code, tokenizer_cobol, max_len_cobol)
    
    # Initialize the decoder input
    decoder_input = np.zeros((1, max_len_summary - 1))
    
    # Predict the summary
    for i in range(max_len_summary - 1):
        output_tokens = model.predict([input_seq, decoder_input])
        sampled_token_index = np.argmax(output_tokens[0, i, :])
        decoder_input[0, i] = sampled_token_index
        
        # Stop if a zero token (padding) is predicted, indicating no more meaningful tokens
        if sampled_token_index == 0:
            break
    
    # Convert token indices back to words
    predicted_summary = []
    for token in decoder_input[0]:
        if token == 0:
            continue
        word = tokenizer_summary.index_word.get(token, '')
        predicted_summary.append(word)
    
    return ' '.join(predicted_summary)


In [ ]:
# Define your COBOL code to be summarized
cobol_code_example = "SORT {DATASET} ON ASCENDING KEY {KEY1} USING {INPUT-FILE} GIVING {OUTPUT-FILE}."

# Predict the summary
summary = predict_summary(cobol_code_example, model, tokenizer_cobol, tokenizer_summary, max_len_cobol, max_len_summary)
print(f'Summary: {summary}')


In [34]:
import pandas as pd

# Load the dataset
df_test = pd.read_csv("C:/Users/Jabasingh Daniel/Desktop/EGDK/Dataset/cob_py.csv")

# Check for NaN values
print(df_test.isna().sum())

# Display rows with NaN values
print(df_test[df_test.isna().any(axis=1)])


cobol_code     0
python_code    1
dtype: int64
                                           cobol_code python_code
23  IDENTIFICATION DIVISION. PROGRAM-ID. PRG18. EN...         NaN


In [48]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Assuming tokenizer_cobol and tokenizer_summary are already fitted on your training data

# Tokenize the COBOL code and Python code
X_test = tokenizer_cobol.texts_to_sequences(df_test['cobol_code'])
Y_test = tokenizer_summary.texts_to_sequences(df_test['python_code'])
